In [69]:
import sys
sys.path.append(str(Path().cwd().parent.parent.resolve()))

import os
from pathlib import Path

import pinecone
import requests
from langchain.document_loaders import (PagedPDFSplitter, TextLoader,
                                        UnstructuredHTMLLoader,
                                        UnstructuredMarkdownLoader,
                                        UnstructuredPDFLoader)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

# from urllib.parse import urlparse

TRUERA_BASE_URL = 'https://truera.com'
TRUREA_DOC_URL = 'https://docs.truera.com/1.34/public'

In [70]:
from concurrent.futures import ThreadPoolExecutor
import datetime
from pathlib import Path
import sqlite3
from url_normalize import url_normalize

from trulens_eval.util import UNICODE_CHECK

class Scrape():
    TABLE_PAGES = "page"

    def __init__(self, filename: Path = Path("scrape.sqlite"), n_threads: int = 8):
        self.filename = filename
        self.db = sqlite3.connect(self.filename)
        self._create_tables()

        self.executor = ThreadPoolExecutor(max_workers = n_threads)

    def _create_tables(self):
        c = self.db.cursor()
        c.execute(f"""
            CREATE TABLE IF NOT EXISTS {Scrape.TABLE_PAGES} (
                url VARCHAR(128),
                type VARCHAR(64),
                retrieved INTEGER,
                content BYTES,
                PRIMARY KEY (url)
            )
        """)
        self.db.commit()
        c.close()
    
    def get_page(self, url: str) -> sqlite3.Row:
        c = self.db.cursor()

        c.execute(f"""
            SELECT * 
            FROM {Scrape.TABLE_PAGES} 
            WHERE url=?""",
            (url,)
        )
        row = c.fetchone()

        c.close()
        return row

    def insert_page(self, url: str, type: str, content: bytes):
        retrieved = datetime.datetime.now().timestamp()

        c = self.db.cursor()

        c.execute(f"""
            INSERT OR REPLACE 
            INTO {Scrape.TABLE_PAGES} 
            VALUES (?, ?, ?, ?)""",
            (url, type, retrieved, content)
        )
        self.db.commit()
        c.close()

        print(f"{UNICODE_CHECK} page {url} -> {self.filename}")

    def scrape(self, url: str):
        url = url_normalize(url)

        page = self.get_page(url)
        if page is not None:
            print("already retrieved")
            return

        print("downloading", end=" ")
        res = requests.get(url)

        if not res.ok:
            raise RuntimeError(res.status_code)

        ctype = res.headers['content-type']

        self.insert_page(url=url, type=ctype, content=res.content)

s = Scrape()
r = s.scrape(TRUERA_BASE_URL)

already retrieved


In [ ]:
from bs4 import BeautifulSoup

# Create a pinecone vector db from a few blogs and docs.
# TODO: langchain includes html loaders which may produce better chunks.

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
pdf_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

scrape_path = Path("webscrape")

collected = dict()
documents = []


def url_to_path(url):
    url_esc = url.replace("https://", "").replace("http://",
                                                  "").replace("/", ":")

    ext = ".html"

    if url_esc.endswith(".png"):
        ext = ""
    elif url_esc.endswith(".pdf"):
        ext = ""
    elif url_esc.endswith(".jpg"):
        ext = ""
    elif url_esc.endswith(".md"):
        ext = ""

    return scrape_path / (url_esc + ext)


def scrape(url):
    if url in collected:
        return

    collected[url] = True

    print(url)

    scrape_file = url_to_path(url)

    if str(url).endswith(".pdf"):
        # skipping for now since issues with the content extractors noted below
        # return
        pass

    if scrape_file.exists():
        print("cached", end=" ")
        content = bytes()
        with scrape_file.open("rb") as fh:
            for line in fh.readlines():
                content += line
    else:
        print("downloading", end=" ")
        response = requests.get(url)

        if response.encoding is None:
            content = response.content

            with scrape_file.open("wb") as fh:
                fh.write(content)

        else:
            content = response.text

            with scrape_file.open("w") as fh:
                fh.write(content)

    loader = UnstructuredHTMLLoader
    if url.endswith(".pdf"):
        #return
        loader = PagedPDFSplitter # freezes for some pdfs
        # loader = UnstructuredPDFLoader # cannot get requirement installation figured out

    elif url.endswith(".png"):
        return
    
    elif url.endswith(".jpg"):
        return
    
    elif url.endswith(".md"):
        loader = UnstructuredMarkdownLoader

    elif (not url.endswith("truera.com")) and (
            not url.endswith("truera.net")) and "." in url[-5:]:
        
        raise RuntimeError(f"Unhandled source type {url}")

    docs = loader(str(scrape_file)).load()
    print(f"got {len(docs)} document(s)")
    for doc in docs:
        doc.metadata['source'] = url
        documents.append(doc)

    try:
        soup = BeautifulSoup(content, 'html.parser')

    except Exception as e:
        print(e)
        return

    for surl in soup.findAll("a"):
        # print(url)
        sub = surl.get('href')
        if sub is not None:
            sub = str(sub)
            # print("\t", sub)

            if sub.startswith("mailto") or sub.startswith("tel"):
                continue

            if not (sub.startswith("http") or sub.startswith("//")):
                sub = url + "/" + sub

            # print("sub=", sub)

            if not (sub.startswith("https://truera.com")
                    or sub.startswith("https://support.truera.com")
                    or sub.startswith("https://marketing.truera.com")
                    or sub.startswith("https://go.truera.com")
                    or sub.startswith("https://app.truera.net")
                    or sub.startswith("https://docs.truera.com")):
                continue

            if "?" in sub:
                continue

            if "#" in sub:
                sub = sub.split("#")[0]

            while "/" == sub[-1]:
                sub = sub[0:-1]

            if sub.endswith("/."):
                continue

            if sub.endswith("/.."):
                continue

            if ".." in sub:
                continue

            if sub.endswith("//"):
                continue

            scrape(sub)

In [ ]:
scrape(TRUERA_BASE_URL)

In [ ]:
#collected = dict()
scrape(TRUREA_DOC_URL)

In [ ]:
scrape("https://truera.com/ai-quality-blog/")

In [ ]:
len(documents)

In [ ]:
# split scraped documents into chunks

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
# keep only big ones

print(len(docs))
bigdocs = [doc for doc in docs if len(doc.page_content) > 256]
print(len(bigdocs))

In [ ]:
from keys import *
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

In [ ]:
# create / upload an index of the docs to pinecone

index_name = "llmdemo"
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims
pinecone.delete_index(index_name)
pinecone.create_index(index_name, dimension=1536)
Pinecone.from_documents(bigdocs, embedding, index_name=index_name)